# 开始

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 100)
# pd.set_option('display.max_columns', 20)
import geohash
from datetime import datetime, time
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

import matplotlib.pyplot as plt
import folium

import sklearn as skl
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
import xgboost as xgb
import lightgbm as lgb
import rcmf as mf
import warnings
warnings.filterwarnings("ignore")

In [2]:
def time_trans(x):
    from datetime import datetime, time
    if type(x) == str:
        x = datetime.strptime(x[11:]+':00', '%H:%M:%S')
        return x.hour * 3600 + x.minute * 60 + x.second
    elif type(x) == int:
        return time(hour=x // 3600, minute=x % 3600 // 60, second=x % 3600 % 60).strftime("%H:%M:%S")
    
# print(time_trans('00:04:20'))
print(time_trans(260))

00:04:20


## 导数据

In [3]:
train_dt = []
for t in range(7):
    train_dt.append(pd.read_csv("D:/Data/toUser/train/训练集_PART"+str(t+1)+".csv"))
train_dt = pd.concat(train_dt).sort_values(['id','visit_time']).reset_index(drop=True)
train_dt.head(2)
print(train_dt.shape)

(43392645, 3)


In [4]:
to_pred_dt = pd.read_csv("D:/Data/toUser/test/测试集.csv")
to_pred_dt.shape

train_dt = train_dt.loc[train_dt.id.isin(to_pred_dt['id'])].reset_index(drop=True)
train_dt.shape

(17270322, 3)

In [5]:
# train_dt = train_dt.loc[train_dt.id.isin(np.random.choice(train_dt.id.unique(), 30000))].sort_values(['id','visit_time']).reset_index(drop=True)
# train_dt.shape

## 妥存ID

In [7]:
id_dict = dict([(v, i) for i, v in enumerate(train_dt.id.unique())])
geo_dict = dict([(v, i) for i, v in enumerate(train_dt.geohash.unique())])

di_dict = dict([(v,k) for k,v in id_dict.items()])
oeg_dict = dict([(v,k) for k,v in geo_dict.items()])

### ID translate

In [8]:
train_dt['id'] = train_dt['id'].map(id_dict)
to_pred_dt['id'] = to_pred_dt['id'].map(id_dict)

1. √ (全量,预备)众数Geo，以及其占比
2. (全量)最后一次Geo值（最大重复序列数占比）
3. (全量)时间最近Geo（knn和置信度）
4. √ (全量,同1)Geo和时间的embedding，Geo的星期分布（均值，方差）
5. 最近两位顺序推断（加最匹配时间）
6. 二分类模型（待定）

## 时间变量

In [9]:
def weekday_calc(x):
    from datetime import datetime, time
    return datetime.weekday(datetime.strptime(x, '%Y-%m-%d %H:%M')) + 1

train_dt['weekday'] = train_dt.visit_time.parallel_apply(weekday_calc)
train_dt['seconds'] = train_dt.visit_time.parallel_apply(time_trans)

## 样本制作

### N-2(train) + 1(validation) + 1(test)样本集

In [102]:
test_set = train_dt.groupby('id').tail(1)
train_set = train_dt.loc[~train_dt.index.isin(test_set.index)]
val_set = train_set.groupby('id').tail(1)
train_set = train_set.loc[~train_set.index.isin(val_set.index)]

print(train_dt.shape, train_set.shape, val_set.shape, test_set.shape)
train_set.head(2)

(17270322, 5) (16870322, 5) (200000, 5) (200000, 5)


,id,visit_time,geohash,weekday,seconds
0,0,2021-10-01 17:06,wm6kfzm,5,61560
1,0,2021-10-02 11:25,wm3tt8q,6,41100


### 验证有多少case，最后一条就在他历史geohash中

In [103]:
pd.merge(val_set[['id','geohash']], train_set[['id','geohash']], 
         on=['id','geohash'], how='inner')['id'].nunique() / len(val_set)

0.77992

In [104]:
pd.merge(test_set[['id','geohash']], train_set[['id','geohash']], 
         on=['id','geohash'], how='inner')['id'].nunique() / len(test_set)

0.774615

# 特征工程

## id 类

### id vs geohash

In [14]:
# geo_embd = train_set[['id','geohash']].value_counts().reset_index()
# geo_embd['geohash'] = geo_embd.geohash.map(geo_dict)
# geo_embd.columns = ['u_id','i_id','rating']

# svd = SVD(lr=0.001, reg=0.005, n_epochs=100, n_factors=5,
#           early_stopping=True, min_rating=1, max_rating=1500)
# svd.fit(X=geo_embd, X_val=geo_embd.sample(10000))

# user_embd = pd.merge(pd.Series(svd.user_mapping_, name='tmp_index'), 
#                      pd.DataFrame(svd.pu_, columns=['geo_embd'+str(i) for i in range(5)]),
#                      left_on='tmp_index', right_index=True)
# del(geo_embd)
# user_embd.head()

### id vs hour

In [15]:
# hour_embd = train_set[['id','visit_time','weekday']] #.value_counts().reset_index()
# hour_embd['weekhour'] = hour_embd.parallel_apply(lambda x:int(x.visit_time[11:13]) * x.weekday,axis=1)
# hour_embd = hour_embd[['id','weekhour']].value_counts().reset_index()
# hour_embd.columns = ['u_id','i_id','rating']

# svd = SVD(lr=0.001, reg=0.005, n_epochs=100, n_factors=5,
#           early_stopping=True, min_rating=1, max_rating=150)
# svd.fit(X=hour_embd, X_val=hour_embd.sample(10000))

# user_embd2 = pd.merge(pd.Series(svd.user_mapping_, name='tmp_index'), 
#                       pd.DataFrame(svd.pu_, columns=['hour_embd'+str(i) for i in range(5)]),
#                       left_on='tmp_index', right_index=True)
# del(hour_embd)
# user_embd2.head()

In [16]:
# hour_embd = train_set[['id','visit_time','geohash']]
# hour_embd['u'], hour_embd['i'] = list(zip(*hour_embd.apply(lambda x:(str(x['id']) + '_' + x['geohash'], int(x.visit_time[11:13])),axis=1)))
# hour_embd = hour_embd.groupby(['u','i'])['id'].count().reset_index()
# hour_embd_umap = dict([(v,i) for i, v in enumerate(hour_embd['u'])])
# hour_embd['u'] = hour_embd['u'].map(hour_embd_umap)
# hour_embd.columns = ['u_id','i_id','rating']

# svd = SVD(lr=0.001, reg=0.005, n_epochs=100, n_factors=2, early_stopping=True, min_rating=1, max_rating=300)
# svd.fit(X=hour_embd, X_val=hour_embd.sample(10000))

# user_embd2 = pd.merge(pd.Series(svd.user_mapping_, name='tmp_index'), 
#                       pd.DataFrame(svd.pu_, columns=['hour_embd'+str(i) for i in range(2)]),
#                       left_on='tmp_index', right_index=True)
# del(hour_embd)

# hour_embd_umap = dict([(v,k) for k,v in hour_embd_umap.items()])
# user_embd2['id'], user_embd2['geohash'] = list(zip(*user_embd2.index.map(lambda x:tuple(hour_embd_umap[x].split('_')))))
# user_embd2['id'] = np.int64(user_embd2['id'])

## id+geo统计

In [105]:
id_geo_stat = train_set.groupby(['id', 'geohash']).agg({'seconds':[np.count_nonzero, np.median, np.mean, np.std], 
                                                        'weekday':[np.median, np.mean, np.std]}).reset_index()

In [ ]:
id_geo_stat.columns = ['id', 'geohash', 'count', 'sec_median', 'sec_mean', 'sec_std', 'week_median', 'week_mean', 'week_std']
id_geo_stat.head()

In [107]:
pred_most_geo = id_geo_stat.sort_values(['id','count']).groupby('id').tail(1)[['id','geohash','count']]
pred_most_geo.columns = ['id','most_geo','most_geo_cnt']

In [108]:
id_geo_stat['id_count'] = id_geo_stat.groupby('id')['count'].transform(sum)
id_geo_stat['id_geo_rate'] = id_geo_stat['count'] / id_geo_stat['id_count']
# id_geo_stat = pd.merge(id_geo_stat, user_embd2.drop('tmp_index',axis=1), on=['id', 'geohash'])

## 最短时间+等待时间

In [109]:
cost_time_dict = dict()

for row in train_set.groupby('id'):
    v = row[1].values
    records_cnt = len(v)
    
    for i in range(records_cnt):
        if i > 0 and v[i][1][:10] == v[i-1][1][:10]:  # same day
            k = tuple(sorted((v[i][2], v[i-1][2])))
            cost_time_dict[k] = cost_time_dict.get(k, []) + [v[i][4] - v[i-1][4]]
            
cost_time_dict = dict([(k, [np.quantile(v, 0.1), len(v), np.std(v)]) for k, v in cost_time_dict.items()])

## 序列性预测

In [110]:
# 序列计算

id_sec_dict = dict(list(zip(*[val_set.id.values, val_set.seconds.values])))  # 将要预测记录的时间（秒）
last_geo_dict = dict([(r[0], r[2]) for r in train_set.sort_values(['id','visit_time']).groupby('id').tail(1).values])  # 已知最后一条的geo
aim_time_dict = dict([(r[0], [r[1], r[-1]]) for r in val_set.values])  # 将要预测记录的时间信息
result_collect = []
result_collect2 = []

for row in train_set.groupby('id'):
    v = row[1].values
    records_cnt = len(v)
    
    dup_cnt = 0
    val_sec = id_sec_dict[row[0]]
    min_sec_geo = [999999, '']  # 逐条比对flag
    
    pair_dict = {}
    pair_time_dict = {}
    for i in range(records_cnt):
        if i > 0 and v[i][2] == v[i-1][2]:
            dup_cnt += 1
        if abs(val_sec - v[i][4]) < min_sec_geo[0]:
            min_sec_geo = [abs(val_sec - v[i][4]), v[i][2]]  # 时间（秒）最近的geo
            
        if i < records_cnt - 1 and v[i][2] == last_geo_dict[row[0]]:  #  沿途拿着last geo比对
            pair_dict[v[i+1][2]] = pair_dict.get(v[i+1][2], 0) + 1
            pair_time_dict[v[i+1][2]] = pair_time_dict.get(v[i+1][2], []) + [(datetime.strptime(v[i+1][1], '%Y-%m-%d %H:%M') - datetime.strptime(v[i][1], '%Y-%m-%d %H:%M')).total_seconds()]  # 时间
    
        if i == records_cnt - 1:  # 最后一条，计算与待预测数据的时间差，是否同一天
            last_geo = v[i][2]  # 已知最后一条的geo
            aim_last_sec = (datetime.strptime(aim_time_dict[row[0]][0], '%Y-%m-%d %H:%M') - datetime.strptime(v[i][1], '%Y-%m-%d %H:%M')).total_seconds()
            same_day = 1 if aim_time_dict[row[0]][0][:10] == v[i][1][:10] else 0
            
    pair_time_dict = dict([(r[0], np.min(np.abs(np.array(r[1]) - aim_last_sec))) for r in pair_time_dict.items()])  #  取最小者
        
    result_collect.append([row[0], min_sec_geo[1], last_geo, dup_cnt, records_cnt, round(dup_cnt/records_cnt, 4)])
    if len(pair_dict) > 0:
        result_collect2.append([row[0], min(pair_time_dict.items(), key=lambda i:i[1])[0], min(pair_time_dict.values()),
                               max(pair_dict.items(), key=lambda x:x[1])[0], max(pair_dict.items(), key=lambda x:x[1])[1],
                               same_day])
    else:
        result_collect2.append([row[0], last_geo_dict[row[0]], 0, last_geo_dict[row[0]], 0, same_day])
    
pred_last_geo = pd.DataFrame(result_collect, columns=['id','nearest_geo','last_geo','dup_cnt','records_cnt','dup_rate'])
pred_next_geo = pd.DataFrame(result_collect2, columns=['id','next_neartime_geo','next_neartime_geo_sec','next_geo','next_geo_cnt','same_day'])

del(result_collect, result_collect2)

## 合并、多分类

In [111]:
val_merge_set = pd.merge(val_set.drop('visit_time',axis=1), pred_most_geo, on='id')
val_merge_set = pd.merge(val_merge_set, pred_last_geo, on='id')
val_merge_set = pd.merge(val_merge_set, pred_next_geo, on='id')

val_merge_set.head(2)

,id,geohash,weekday,seconds,most_geo,most_geo_cnt,nearest_geo,last_geo,dup_cnt,records_cnt,dup_rate,next_neartime_geo,next_neartime_geo_sec,next_geo,next_geo_cnt,same_day
0,0,wm3yx3t,7,58740,wm6n2jm,49,wm3yx9y,wm6n2jm,33,83,0.3976,wm6n2jm,960.0,wm6n2jm,31,1
1,1,wm3vz92,6,81000,wm3vz92,36,wm3vz92,wm3vz92,26,91,0.2857,wm3vz98,3900.0,wm3vz92,15,1


### 做标签、做剩余特征

In [112]:
def tmp(x):
    bingo_list = []
    if x.geohash == x.most_geo:
        bingo_list.append(1)
    if x.geohash == x.nearest_geo:
        bingo_list.append(2)
    if x.geohash == x.next_neartime_geo:
        bingo_list.append(5)

    return bingo_list, round(x.most_geo_cnt / x.records_cnt, 4), round(x.next_geo_cnt / x.records_cnt, 4)
    
val_merge_set['bingo_list'], val_merge_set['most_geo_rate'], val_merge_set['next_geo_rate'] = list(zip(*val_merge_set.apply(tmp, axis=1)))

In [ ]:
val_merge_set['bingo_list'].value_counts()
# 1 2 5

In [ ]:
[]           90951
[1, 2, 5]    30132
[1, 2]       15594
[2]          15385
[5]          14071
[2, 5]       13776
[1]          11167
[1, 5]        8924

In [113]:
# 二阶段上限准确率
sum(val_merge_set.bingo_list.map(lambda x:1 if len(x) > 0 else 0)) / 200000

0.545245

1 2 5: 0.545245

In [114]:
val_merge_set['last_most_min_sec'],val_merge_set['last_most_min_cnt'],val_merge_set['last_most_min_std'] = list(zip(\
    *val_merge_set.apply(lambda x:cost_time_dict.get(tuple(sorted((x.last_geo, x.most_geo))),[0,0,0]),axis=1)))
val_merge_set['last_nearest_min_sec'],val_merge_set['last_nearest_min_cnt'],val_merge_set['last_nearest_min_std'] = list(zip(\
    *val_merge_set.apply(lambda x:cost_time_dict.get(tuple(sorted((x.last_geo, x.nearest_geo))),[0,0,0]),axis=1)))
val_merge_set['last_neartime_min_sec'],val_merge_set['last_neartime_min_cnt'],val_merge_set['last_neartime_min_std'] = list(zip(\
    *val_merge_set.apply(lambda x:cost_time_dict.get(tuple(sorted((x.last_geo, x.next_neartime_geo))),[0,0,0]),axis=1)))

In [115]:
del(cost_time_dict)

In [116]:
val_merge_set = pd.merge(val_merge_set, id_geo_stat.rename({**{'geohash':'most_geo'}, **dict([(x, 'most_geo_'+x) for x in id_geo_stat.columns[2:]])},axis=1), on=['id','most_geo'])
val_merge_set = pd.merge(val_merge_set, id_geo_stat.rename({**{'geohash':'nearest_geo'}, **dict([(x, 'nearest_geo_'+x) for x in id_geo_stat.columns[2:]])},axis=1), on=['id','nearest_geo'])
val_merge_set = pd.merge(val_merge_set, id_geo_stat.rename({**{'geohash':'next_neartime_geo'}, **dict([(x, 'next_neartime_geo_'+x) for x in id_geo_stat.columns[2:]])},axis=1), on=['id','next_neartime_geo'])
# val_merge_set = pd.merge(val_merge_set, id_geo_stat.rename({**{'geohash':'next_geo'}, **dict([(x, 'next_geo_'+x) for x in id_geo_stat.columns[2:]])},axis=1), on=['id','next_geo'])

### 再补充一些最终时间的差值特征

In [117]:
# parallel_apply
val_merge_set['most_week_diff_median'] = val_merge_set.apply(lambda x:abs(x.weekday - x.most_geo_week_median), axis=1)
val_merge_set['nearest_week_diff_median'] = val_merge_set.apply(lambda x:abs(x.weekday - x.nearest_geo_week_median), axis=1)
val_merge_set['neartime_week_diff_median'] = val_merge_set.apply(lambda x:abs(x.weekday - x.next_neartime_geo_week_median), axis=1)

val_merge_set['most_sec_diff_median'] = val_merge_set.apply(lambda x:abs(x.seconds - x.most_geo_sec_median), axis=1)
val_merge_set['nearest_sec_diff_median'] = val_merge_set.apply(lambda x:abs(x.seconds - x.nearest_geo_sec_median), axis=1)
val_merge_set['neartime_sec_diff_median'] = val_merge_set.apply(lambda x:abs(x.seconds - x.next_neartime_geo_sec_median), axis=1)

In [118]:
val_merge_set['most_week_diff_mean'] = val_merge_set.apply(lambda x:abs(x.weekday - x.most_geo_week_mean), axis=1)
val_merge_set['nearest_week_diff_mean'] = val_merge_set.apply(lambda x:abs(x.weekday - x.nearest_geo_week_mean), axis=1)
val_merge_set['neartime_week_diff_mean'] = val_merge_set.apply(lambda x:abs(x.weekday - x.next_neartime_geo_week_mean), axis=1)

val_merge_set['most_sec_diff_mean'] = val_merge_set.apply(lambda x:abs(x.seconds - x.most_geo_sec_mean), axis=1)
val_merge_set['nearest_sec_diff_mean'] = val_merge_set.apply(lambda x:abs(x.seconds - x.nearest_geo_sec_mean), axis=1)
val_merge_set['neartime_sec_diff_mean'] = val_merge_set.apply(lambda x:abs(x.seconds - x.next_neartime_geo_sec_mean), axis=1)

### 一些最终距离的差值特征

In [45]:
def calc_distance(row):
    from math import acos, asin, cos, sin, sqrt, pi
    import geohash
            
    ap_latlon, bp_latlon = row
    
    ap_latlon = geohash.decode(ap_latlon)
    bp_latlon = geohash.decode(bp_latlon)
        
    radLat1 = ap_latlon[0] * pi / 180.0;
    radLat2 = bp_latlon[0] * pi / 180.0;
    diffLat = radLat1 - radLat2;
    
    radLng1 = ap_latlon[1] * pi / 180.0;
    radLng2 = bp_latlon[1] * pi / 180.0;
    diffLng = radLng1 - radLng2;
    
    dist = 2 * asin(sqrt(pow(sin(diffLat / 2), 2) + cos(radLat1) * cos(radLat2) * pow(sin(diffLng / 2), 2))) * 6371000

    return dist

In [119]:
val_merge_set['most_geo_dist'] = val_merge_set[['last_geo','most_geo']].apply(calc_distance, axis=1)
val_merge_set['nearest_geo_dist'] = val_merge_set[['last_geo','nearest_geo']].apply(calc_distance, axis=1)
val_merge_set['next_neartime_geo_dist'] = val_merge_set[['last_geo','next_neartime_geo']].apply(calc_distance, axis=1)

In [120]:
val_merge_set['most_geo_speed_median'] = val_merge_set.apply(lambda x:-1 if x.most_sec_diff_median == 0 else x.most_geo_dist / x.most_sec_diff_median,axis=1)
val_merge_set['nearest_geo_speed_median'] = val_merge_set.apply(lambda x:-1 if x.nearest_sec_diff_median == 0 else x.nearest_geo_dist / x.nearest_sec_diff_median,axis=1)
val_merge_set['next_neartime_geo_speed_median'] = val_merge_set.apply(lambda x:-1 if x.neartime_sec_diff_median == 0 else x.next_neartime_geo_dist / x.neartime_sec_diff_median,axis=1)

In [121]:
val_merge_set['most_geo_speed_mean'] = val_merge_set.apply(lambda x:-1 if x.most_sec_diff_mean == 0 else x.most_geo_dist / x.most_sec_diff_mean,axis=1)
val_merge_set['nearest_geo_speed_mean'] = val_merge_set.apply(lambda x:-1 if x.nearest_sec_diff_mean == 0 else x.nearest_geo_dist / x.nearest_sec_diff_mean,axis=1)
val_merge_set['next_neartime_geo_speed_mean'] = val_merge_set.apply(lambda x:-1 if x.neartime_sec_diff_mean == 0 else x.next_neartime_geo_dist / x.neartime_sec_diff_mean,axis=1)

In [49]:
val_merge_set.columns

Index(['id', 'geohash', 'weekday', 'seconds', 'most_geo', 'most_geo_cnt',
       'nearest_geo', 'last_geo', 'dup_cnt', 'records_cnt', 'dup_rate',
       'next_neartime_geo', 'next_neartime_geo_sec', 'next_geo',
       'next_geo_cnt', 'same_day', 'bingo_list', 'most_geo_rate',
       'next_geo_rate', 'last_most_min_sec', 'last_most_min_cnt',
       'last_most_min_std', 'last_nearest_min_sec', 'last_nearest_min_cnt',
       'last_nearest_min_std', 'last_neartime_min_sec',
       'last_neartime_min_cnt', 'last_neartime_min_std', 'most_geo_count',
       'most_geo_sec_median', 'most_geo_sec_mean', 'most_geo_sec_std',
       'most_geo_week_median', 'most_geo_week_mean', 'most_geo_week_std',
       'most_geo_id_count', 'most_geo_id_geo_rate', 'nearest_geo_count',
       'nearest_geo_sec_median', 'nearest_geo_sec_mean', 'nearest_geo_sec_std',
       'nearest_geo_week_median', 'nearest_geo_week_mean',
       'nearest_geo_week_std', 'nearest_geo_id_count',
       'nearest_geo_id_geo_rate', 'nex

In [122]:
choice_set = val_merge_set.explode('bingo_list').query('bingo_list > 0').drop(['id','geohash','nearest_geo','next_neartime_geo','most_geo','last_geo','next_geo'],axis=1)

### Ensemble

In [82]:
# model = xgb.XGBClassifier(objective='multi:softprob', colsample_bytree=0.9, learning_rate=0.3, n_estimators = 90, max_depth = 2, random_state = 42)

# # 设置网格参数
# params = [{'n_estimators': [80, 75, 70], 'colsample_bytree': [0.85, 0.8], 'learning_rate':[0.1, 0.15, 0.2]}]
# model = GridSearchCV(model, params, cv=3)
# model.fit(choice_set.drop(['bingo_list'],axis=1), choice_set.bingo_list, eval_metric='mlogloss')

# print(model.best_params_)   # 最优参数
# print(model.best_score_)    # 最好得分
# print(model.best_estimator_)

{'colsample_bytree': 0.85, 'learning_rate': 0.1, 'n_estimators': 80}
nan
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.85, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=2,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=80, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', random_state=42, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)


In [ ]:
def train_func(params, dt=val_merge_set):
    kf = KFold(n_splits=3)
    kf.get_n_splits(choice_set)
    
    for i, (train_index, test_index) in enumerate(kf.split(choice_set)):
        X_train, X_test = choice_set.drop(['bingo_list'],axis=1).iloc[train_index], choice_set.drop(['bingo_list'],axis=1).iloc[test_index]
        y_train, y_test = choice_set['bingo_list'].iloc[train_index], choice_set['bingo_list'].iloc[test_index]
        
        model = xgb.XGBClassifier(**params, colsample_bytree = 0.85, learning_rate = 0.4, objective='multi:softprob', )
        model.fit(X_train.values, y_train, eval_metric='mlogloss')
        model_class_dict = dict([(i, x) for i, x in enumerate(model.classes_)])
        
        train_acc = sum(np.array([model_class_dict[list(x).index(max(x))] for x in model.predict_proba(X_train)]) == y_train) / len(y_train)
        test_acc = sum(np.array([model_class_dict[list(x).index(max(x))] for x in model.predict_proba(X_test)]) == y_test) / len(y_test)
    
        dt['pred'] = np.array([model_class_dict[list(x).index(max(x))] for x in model.predict_proba(dt[X_train.columns])])
    
        # 1 2 5 0.46114
        val_actu_acc = sum(dt.apply(lambda x:1 if (x.pred == 1 and x.geohash == x.most_geo) or (x.pred == 2 and x.geohash == x.nearest_geo) or (x.pred == 5 and x.geohash == x.next_neartime_geo) else 0,axis=1)) / 200000
        dt = dt.drop('pred', axis=1)
        
    print(np.mean(train_acc), np.mean(test_acc), np.mean(val_actu_acc))
    
for m in [5]:
    for n in (200, 180):
        params = {'max_depth':m, 'n_estimators':n}
        print(params)
        train_func(params)

In [125]:
X_train, X_test, y_train, y_test = train_test_split(choice_set.drop(['bingo_list'],axis=1), choice_set.bingo_list, test_size=0.1)
model = xgb.XGBClassifier(objective='multi:softprob', colsample_bytree=0.85, learning_rate=0.4, n_estimators = 180, max_depth = 4, random_state = 5)
model.fit(X_train, y_train)
model_class_dict = dict([(i, x) for i, x in enumerate(model.classes_)])

[14:34:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.48 0.47 objective='multi:softprob', colsample_bytree=0.85, learning_rate=0.3, n_estimators = 90, max_depth = 2, random_state = 42

0.45 0.43 colsample_bytree=1, learning_rate=0.3, n_estimators = 90 

0.45 0.42 colsample_bytree=0.85, learning_rate=0.4, n_estimators = 180

In [126]:
print('train acc', sum(np.array([model_class_dict[list(x).index(max(x))] for x in model.predict_proba(X_train)]) == y_train) / len(y_train))
print('test acc', sum(np.array([model_class_dict[list(x).index(max(x))] for x in model.predict_proba(X_test)]) == y_test) / len(y_test))

val_merge_set['pred'] = np.array([model_class_dict[list(x).index(max(x))] for x in model.predict_proba(val_merge_set[X_train.columns])])

# 1 2 5 0.465435
print('val actu acc', sum(val_merge_set.apply(lambda x:1 if (x.pred == 1 and x.geohash == x.most_geo) or (x.pred == 2 and x.geohash == x.nearest_geo) or (x.pred == 5 and x.geohash == x.next_neartime_geo) else 0,axis=1)) / 200000)
val_merge_set = val_merge_set.drop('pred', axis=1)

train acc 0.47505967481241235
test acc 0.3693945378353644
val actu acc 0.48216


In [127]:
fis = pd.DataFrame(list(zip(X_train.columns.values, model.feature_importances_)), columns=['cols','fi'])
fis.sort_values('fi', ascending = False)

,cols,fi
56,most_sec_diff_mean,0.091634
38,next_neartime_geo_count,0.070278
46,next_neartime_geo_id_geo_rate,0.058399
29,nearest_geo_count,0.053357
8,same_day,0.050484
37,nearest_geo_id_geo_rate,0.042693
58,neartime_sec_diff_mean,0.038537
57,nearest_sec_diff_mean,0.037546
52,neartime_sec_diff_median,0.032140
50,most_sec_diff_median,0.031145


In [128]:
model_cols = X_train.columns
del(X_train, X_test, y_train, y_test)

In [129]:
model.save_model('D:Desktop/时空预测算法对抗赛/final_model.model')

# Bad case

In [61]:
bad_case_id = val_merge_set.loc[val_merge_set.bingo_list.map(lambda x:len(x) == 0), 'id']

In [105]:
pd.merge(id_geo_stat[['id','geohash','count','id_geo_rate']], val_set.loc[val_set.id.isin(bad_case_id), ['id','geohash']],
         on=['id','geohash'], how='inner').to_csv('D:Desktop/bad case.csv', index=False)

In [62]:
bad_case_id.map(di_dict).to_csv('D:Desktop/bad case.csv', index=False)

In [63]:
val_merge_set.head(2)

,id,geohash,weekday,seconds,most_geo,most_geo_cnt,nearest_geo,last_geo,dup_cnt,records_cnt,...,most_geo_dist,nearest_geo_dist,next_neartime_geo_dist,most_geo_speed_median,nearest_geo_speed_median,next_neartime_geo_speed_median,most_geo_speed_mean,nearest_geo_speed_mean,next_neartime_geo_speed_mean,pred
0,0,wm3yx3t,7,58740,wm6n2jm,49,wm3yx9y,wm6n2jm,33,83,...,0.0,3381.777133,0.0,0.0,4.335612,0.0,0.0,0.650342,0.0,5
1,1,wm3vz92,6,81000,wm3vz92,36,wm3vz92,wm3vz92,26,91,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,2


In [64]:
val_merge_set.columns

Index(['id', 'geohash', 'weekday', 'seconds', 'most_geo', 'most_geo_cnt',
       'nearest_geo', 'last_geo', 'dup_cnt', 'records_cnt', 'dup_rate',
       'next_neartime_geo', 'next_neartime_geo_sec', 'next_geo',
       'next_geo_cnt', 'same_day', 'bingo_list', 'most_geo_rate',
       'next_geo_rate', 'most_geo_count', 'most_geo_sec_median',
       'most_geo_sec_mean', 'most_geo_sec_std', 'most_geo_week_median',
       'most_geo_week_mean', 'most_geo_week_std', 'most_geo_id_count',
       'most_geo_id_geo_rate', 'nearest_geo_count', 'nearest_geo_sec_median',
       'nearest_geo_sec_mean', 'nearest_geo_sec_std',
       'nearest_geo_week_median', 'nearest_geo_week_mean',
       'nearest_geo_week_std', 'nearest_geo_id_count',
       'nearest_geo_id_geo_rate', 'next_neartime_geo_count',
       'next_neartime_geo_sec_median', 'next_neartime_geo_sec_mean',
       'next_neartime_geo_sec_std', 'next_neartime_geo_week_median',
       'next_neartime_geo_week_mean', 'next_neartime_geo_week_std',
  

In [125]:
val_sample = val_merge_set.loc[val_merge_set['id'] == bad_case_id.sample(1).values[0] ,['id','geohash','seconds','most_geo','nearest_geo','next_neartime_geo']].to_dict('records')[0]
val_sample = val_merge_set.loc[val_merge_set['id'] == 56917 ,['id','geohash','seconds','most_geo','nearest_geo','next_neartime_geo']].to_dict('records')[0]
train_sample = train_set.loc[train_set['id'] == val_sample['id']]
train_sample['lat'], train_sample['lng'] = list(zip(*train_sample['geohash'].map(geohash.decode)))

incidents = folium.map.FeatureGroup()
san_map = folium.Map(location=geohash.decode(val_sample['geohash']), zoom_start=13.5, 
                     tiles='http://webrd02.is.autonavi.com/appmaptile?lang=zh_cn&size=1&scale=1&style=7&x={x}&y={y}&z={z}',
                     attr='default')
def utf2asc(s):
    return str(str(s).encode('ascii', 'xmlcharrefreplace'))[2:-1]

# 历史poi坐标点
def point_in1(p):   
    incidents.add_child(
        folium.CircleMarker(
            [p.lat, p.lng],
            radius=4,  
            color='black',  # color_list[color_map[p.waybill_code]],  # color_list.get(p.dbsc, 'white'),  
            fill=True,
            fill_opacity=0.8,
            tooltip=utf2asc(p.geohash + ' ' + p.visit_time),
        )
    )

    
incidents.add_child(folium.CircleMarker(geohash.decode(val_sample['geohash']), color='red', tooltip=utf2asc('True'), radius=4, fill=True, fill_opacity=0.8,))

train_sample[['id','visit_time','geohash','lat','lng']].apply(point_in1, axis = 1)

incidents.add_child(folium.CircleMarker(geohash.decode(val_sample['most_geo']), color='blue', tooltip=utf2asc('most_geo:'+val_sample['most_geo']), radius=4, fill=True, fill_opacity=0.8,))
incidents.add_child(folium.CircleMarker(geohash.decode(val_sample['nearest_geo']), color='green', tooltip=utf2asc('nearest_geo:'+val_sample['nearest_geo']), radius=4, fill=True, fill_opacity=0.8,))
incidents.add_child(folium.CircleMarker(geohash.decode(val_sample['next_neartime_geo']), color='yellow', tooltip=utf2asc('next_neartime_geo:'+val_sample['next_neartime_geo']), radius=4, fill=True, fill_opacity=0.8,))

san_map.add_child(incidents)


# 连接线 
train_sample_v = train_sample.values
for i in range(len(train_sample_v)-1):
    if i >= len(train_sample_v) - 5:
        folium.PolyLine([[train_sample_v[i][5], train_sample_v[i][6]], 
                         [train_sample_v[i+1][5], train_sample_v[i+1][6]]] ,color='black' ).add_to(san_map)
    
folium.PolyLine([geohash.decode(val_sample['geohash']), geohash.decode(val_sample['most_geo'])] ,color='blue', tooltip='most_geo').add_to(san_map)
folium.PolyLine([geohash.decode(val_sample['geohash']), geohash.decode(val_sample['nearest_geo'])] ,color='green', tooltip='nearest_geo').add_to(san_map)
folium.PolyLine([geohash.decode(val_sample['geohash']), geohash.decode(val_sample['next_neartime_geo'])] ,color='yellow', tooltip='next_neartime_geo').add_to(san_map)
    
# folium.vector_layers.PolyLine([geohash.decode(val_sample['geohash']), geohash.decode(val_sample['next_neartime_geo'])] ,color='black', tooltip='next_neartime_geo').add_to(san_map)    
    
san_map

In [126]:
val_sample

{'id': 56917,
 'geohash': 'wm3yr2h',
 'seconds': 71400,
 'most_geo': 'wm6j8ns',
 'nearest_geo': 'wm3yx3y',
 'next_neartime_geo': 'wm6j8nf'}

In [ ]:
pd.concat([train_sample.tail(30), val_set.loc[val_set['id'] == val_sample['id']]])

In [ ]:
train_sample.groupby('geohash')['seconds','weekday'].describe()

In [78]:
coll = dict()
flag = 0
for r in train_set.values:
    if r[2] == 'wm6jueq':
        flag = 1
        i = r[1][:10]
        j = r[4]
        continue
    if flag == 1:
        flag = 0
        if i == r[1][:10]:
            coll[r[2]] = coll.get(r[2], []) + [r[4] - j]

In [83]:
coll = [(i[0], len(i[1]), np.mean(i[1]), np.median(i[1])) for i in coll.items()]

In [86]:
pd.DataFrame(coll, columns=['geo','cnt','mean','median']).to_csv('D:Desktop/tmp.csv')

In [ ]:
train_sample.groupby('geohash')[['weekday','seconds']].describe()

# 打分过程

In [ ]:
# model = xgb.Booster(model_file='D:Desktop/时空预测算法对抗赛/final_model.xgb')
# model.predict(val_merge_set[X_train.columns])

In [130]:
train_set = train_dt.copy()

id_geo_stat = train_set.groupby(['id', 'geohash']).agg({'seconds':[np.count_nonzero, np.median, np.mean, np.std], 
                                                        'weekday':[np.median, np.mean, np.std]}).reset_index()

id_geo_stat.columns = ['id', 'geohash', 'count', 'sec_median', 'sec_mean', 'sec_std', 'week_median', 'week_mean', 'week_std']

pred_most_geo = id_geo_stat.sort_values(['id','count']).groupby('id').tail(1)[['id','geohash','count']]
pred_most_geo.columns = ['id','most_geo','most_geo_cnt']

id_geo_stat['id_count'] = id_geo_stat.groupby('id')['count'].transform(sum)
id_geo_stat['id_geo_rate'] = id_geo_stat['count'] / id_geo_stat['id_count']

to_pred_dt['weekday'] = to_pred_dt.visit_time.apply(weekday_calc)
to_pred_dt['seconds'] = to_pred_dt.visit_time.apply(time_trans)

cost_time_dict = dict()

for row in train_set.groupby('id'):
    v = row[1].values
    records_cnt = len(v)
    
    for i in range(records_cnt):
        if i > 0 and v[i][1][:10] == v[i-1][1][:10]:  # same day
            k = tuple(sorted((v[i][2], v[i-1][2])))
            cost_time_dict[k] = cost_time_dict.get(k, []) + [v[i][4] - v[i-1][4]]

cost_time_dict = dict([(k, [np.quantile(v, 0.1), len(v), np.std(v)]) for k, v in cost_time_dict.items()])



# 大循环
id_sec_dict = dict(list(zip(*[to_pred_dt.id.values, to_pred_dt.seconds.values])))  # 将要预测记录的时间（秒）
last_geo_dict = dict([(r[0], r[2]) for r in train_set.sort_values(['id','visit_time']).groupby('id').tail(1).values])  # 已知最后一条的geo
aim_time_dict = dict([(r[0], [r[1], r[-1]])for r in to_pred_dt.values])  # 将要预测记录的时间信息
result_collect = []
result_collect2 = []

for row in train_set.groupby('id'):
    v = row[1].values
    records_cnt = len(v)
    
    dup_cnt = 0
    val_sec = id_sec_dict[row[0]]
    min_sec_geo = [999999, '']
    
    pair_dict = {}
    pair_time_dict = {}
    for i in range(records_cnt):
        if i > 0 and v[i][2] == v[i-1][2]:
            dup_cnt += 1
        if abs(val_sec - v[i][4]) < min_sec_geo[0]:
            min_sec_geo = [abs(val_sec - v[i][4]), v[i][2]]  # 时间（秒）最近的geo
            
        if i < records_cnt - 1 and v[i][2] == last_geo_dict[row[0]]:  #  沿途拿着last geo比对
            pair_dict[v[i+1][2]] = pair_dict.get(v[i+1][2], 0) + 1
            pair_time_dict[v[i+1][2]] = pair_time_dict.get(v[i+1][2], []) + [(datetime.strptime(v[i+1][1], '%Y-%m-%d %H:%M') - datetime.strptime(v[i][1], '%Y-%m-%d %H:%M')).total_seconds()]  # 时间
    
        if i == records_cnt - 1:  # 最后一条，计算与待预测数据的时间差，是否同一天
            last_geo = v[i][2]  # 已知最后一条的geo
            aim_last_sec = (datetime.strptime(aim_time_dict[row[0]][0], '%Y-%m-%d %H:%M') - datetime.strptime(v[i][1], '%Y-%m-%d %H:%M')).total_seconds()
            same_day = 1 if aim_time_dict[row[0]][0][:10] == v[i][1][:10] else 0
            
    pair_time_dict = dict([(r[0], np.min(np.abs(np.array(r[1]) - aim_last_sec))) for r in pair_time_dict.items()])  #  取最小者
        
    result_collect.append([row[0], min_sec_geo[1], last_geo, dup_cnt, records_cnt, round(dup_cnt/records_cnt, 4)])
    if len(pair_dict) > 0:
        result_collect2.append([row[0], min(pair_time_dict.items(), key=lambda i:i[1])[0], min(pair_time_dict.values()),
                               max(pair_dict.items(), key=lambda x:x[1])[0], max(pair_dict.items(), key=lambda x:x[1])[1],
                               same_day])
    else:
        result_collect2.append([row[0], last_geo_dict[row[0]], 0, last_geo_dict[row[0]], 0, same_day])
    
pred_last_geo = pd.DataFrame(result_collect, columns=['id','nearest_geo','last_geo','dup_cnt','records_cnt','dup_rate'])
pred_next_geo = pd.DataFrame(result_collect2, columns=['id','next_neartime_geo','next_neartime_geo_sec','next_geo','next_geo_cnt','same_day'])

del(result_collect, result_collect2)

## 合并、多分类

val_merge_set = pd.merge(to_pred_dt.drop('visit_time',axis=1), pred_most_geo, on='id')
val_merge_set = pd.merge(val_merge_set, pred_last_geo, on='id')
val_merge_set = pd.merge(val_merge_set, pred_next_geo, on='id')

### 做标签、做剩余特征

def tmp(x): 
    return round(x.most_geo_cnt / x.records_cnt, 4), round(x.next_geo_cnt / x.records_cnt, 4)
    
val_merge_set['most_geo_rate'], val_merge_set['next_geo_rate'] = list(zip(*val_merge_set.apply(tmp, axis=1)))

val_merge_set['last_most_min_sec'],val_merge_set['last_most_min_cnt'],val_merge_set['last_most_min_std'] = list(zip(\
    *val_merge_set.apply(lambda x:cost_time_dict.get(tuple(sorted((x.last_geo, x.most_geo))),[0,0,0]),axis=1)))
val_merge_set['last_nearest_min_sec'],val_merge_set['last_nearest_min_cnt'],val_merge_set['last_nearest_min_std'] = list(zip(\
    *val_merge_set.apply(lambda x:cost_time_dict.get(tuple(sorted((x.last_geo, x.nearest_geo))),[0,0,0]),axis=1)))
val_merge_set['last_neartime_min_sec'],val_merge_set['last_neartime_min_cnt'],val_merge_set['last_neartime_min_std'] = list(zip(\
    *val_merge_set.apply(lambda x:cost_time_dict.get(tuple(sorted((x.last_geo, x.next_neartime_geo))),[0,0,0]),axis=1)))

del(cost_time_dict)

val_merge_set = pd.merge(val_merge_set, id_geo_stat.rename({**{'geohash':'most_geo'}, **dict([(x, 'most_geo_'+x) for x in id_geo_stat.columns[2:]])},axis=1), on=['id','most_geo'])
val_merge_set = pd.merge(val_merge_set, id_geo_stat.rename({**{'geohash':'nearest_geo'}, **dict([(x, 'nearest_geo_'+x) for x in id_geo_stat.columns[2:]])},axis=1), on=['id','nearest_geo'])
val_merge_set = pd.merge(val_merge_set, id_geo_stat.rename({**{'geohash':'next_neartime_geo'}, **dict([(x, 'next_neartime_geo_'+x) for x in id_geo_stat.columns[2:]])},axis=1), on=['id','next_neartime_geo'])


val_merge_set['most_week_diff_median'] = val_merge_set.apply(lambda x:abs(x.weekday - x.most_geo_week_median), axis=1)
val_merge_set['nearest_week_diff_median'] = val_merge_set.apply(lambda x:abs(x.weekday - x.nearest_geo_week_median), axis=1)
val_merge_set['neartime_week_diff_median'] = val_merge_set.apply(lambda x:abs(x.weekday - x.next_neartime_geo_week_median), axis=1)

val_merge_set['most_sec_diff_median'] = val_merge_set.apply(lambda x:abs(x.seconds - x.most_geo_sec_median), axis=1)
val_merge_set['nearest_sec_diff_median'] = val_merge_set.apply(lambda x:abs(x.seconds - x.nearest_geo_sec_median), axis=1)
val_merge_set['neartime_sec_diff_median'] = val_merge_set.apply(lambda x:abs(x.seconds - x.next_neartime_geo_sec_median), axis=1)

val_merge_set['most_week_diff_mean'] = val_merge_set.apply(lambda x:abs(x.weekday - x.most_geo_week_mean), axis=1)
val_merge_set['nearest_week_diff_mean'] = val_merge_set.apply(lambda x:abs(x.weekday - x.nearest_geo_week_mean), axis=1)
val_merge_set['neartime_week_diff_mean'] = val_merge_set.apply(lambda x:abs(x.weekday - x.next_neartime_geo_week_mean), axis=1)

val_merge_set['most_sec_diff_mean'] = val_merge_set.apply(lambda x:abs(x.seconds - x.most_geo_sec_mean), axis=1)
val_merge_set['nearest_sec_diff_mean'] = val_merge_set.apply(lambda x:abs(x.seconds - x.nearest_geo_sec_mean), axis=1)
val_merge_set['neartime_sec_diff_mean'] = val_merge_set.apply(lambda x:abs(x.seconds - x.next_neartime_geo_sec_mean), axis=1)

def calc_distance(row):
    from math import acos, asin, cos, sin, sqrt, pi
    import geohash
            
    ap_latlon, bp_latlon = row
    
    ap_latlon = geohash.decode(ap_latlon)
    bp_latlon = geohash.decode(bp_latlon)
        
    radLat1 = ap_latlon[0] * pi / 180.0;
    radLat2 = bp_latlon[0] * pi / 180.0;
    diffLat = radLat1 - radLat2;
    
    radLng1 = ap_latlon[1] * pi / 180.0;
    radLng2 = bp_latlon[1] * pi / 180.0;
    diffLng = radLng1 - radLng2;
    
    dist = 2 * asin(sqrt(pow(sin(diffLat / 2), 2) + cos(radLat1) * cos(radLat2) * pow(sin(diffLng / 2), 2))) * 6371000

    return dist

val_merge_set['most_geo_dist'] = val_merge_set[['last_geo','most_geo']].apply(calc_distance, axis=1)
val_merge_set['nearest_geo_dist'] = val_merge_set[['last_geo','nearest_geo']].apply(calc_distance, axis=1)
val_merge_set['next_neartime_geo_dist'] = val_merge_set[['last_geo','next_neartime_geo']].apply(calc_distance, axis=1)

val_merge_set['most_geo_speed_median'] = val_merge_set.apply(lambda x:-1 if x.most_sec_diff_median == 0 else x.most_geo_dist / x.most_sec_diff_median,axis=1)
val_merge_set['nearest_geo_speed_median'] = val_merge_set.apply(lambda x:-1 if x.nearest_sec_diff_median == 0 else x.nearest_geo_dist / x.nearest_sec_diff_median,axis=1)
val_merge_set['next_neartime_geo_speed_median'] = val_merge_set.apply(lambda x:-1 if x.neartime_sec_diff_median == 0 else x.next_neartime_geo_dist / x.neartime_sec_diff_median,axis=1)

val_merge_set['most_geo_speed_mean'] = val_merge_set.apply(lambda x:-1 if x.most_sec_diff_mean == 0 else x.most_geo_dist / x.most_sec_diff_mean,axis=1)
val_merge_set['nearest_geo_speed_mean'] = val_merge_set.apply(lambda x:-1 if x.nearest_sec_diff_mean == 0 else x.nearest_geo_dist / x.nearest_sec_diff_mean,axis=1)
val_merge_set['next_neartime_geo_speed_mean'] = val_merge_set.apply(lambda x:-1 if x.neartime_sec_diff_mean == 0 else x.next_neartime_geo_dist / x.neartime_sec_diff_mean,axis=1)

pred_out = np.array([model_class_dict[list(x).index(max(x))] for x in model.predict_proba(val_merge_set[model_cols])])

val_merge_set['pred_out'] = pred_out

def tmp(x):
    if x.pred_out == 1:
        return x.most_geo
    elif x.pred_out == 2:
        return x.nearest_geo
    elif x.pred_out == 5:
        return x.next_neartime_geo
    
val_merge_set['geohash'] = val_merge_set.apply(tmp,axis=1)

x = pd.merge(to_pred_dt[['id','visit_time']], val_merge_set[['id','geohash']], on='id')
x['id'] = x['id'].map(di_dict)
x[['id','visit_time','geohash']].to_csv("d:Desktop/output_0820_3.csv", encoding='UTF-8', sep=',', index=False, header=True)